In [1]:
# Importing required modules
import json

from prettytable import PrettyTable
from stringMatcher import levenshteinDistanceRatio
from time import time

# Making an object of prettytable
table = PrettyTable(field_names=["SpectatorID", "Out", "Number Of Comments", "Average", "Number Of Balls"])

In [2]:
#Loading database in the memory
data = json.load(open("dataKeeper/dataBase.json", "rb"))

In [3]:
#Loading global secondary index in the memory
GSI = json.load(open("dataKeeper/dataBase_GSI.json", "rb"))

In [4]:
# Creating a function to get the closest match of searchString
def getClosestMatch(searchString):
    # Initializing the time counter
    tic = time()
    
    # Finding closest match of the searchString from the Global Secondary Index
    matchRatios = [levenshteinDistanceRatio(searchString, KEY) for KEY, _ in GSI.items()]
    bestMatchRatio = max(matchRatios)

    # Match ratio < 50% => No record
    if (bestMatchRatio < 0.5):
        return {
            "response code" : 404,
            "search time"   : f"{time()-tic} seconds",
            "message"       : "error",
            "response body" : "no match found"
        }
    
    # Match ratio >= 50% => Follow the code below
    bestMatchRatio_index = matchRatios.index(bestMatchRatio)

    # Finding best match for searchString
    bestMatch_PlayerName = list(GSI)[bestMatchRatio_index]
    bestMatch_PlayerId = GSI[bestMatch_PlayerName]

    # Using playerId as primary key, retrieving the details 
    playerInfo = data[str(bestMatch_PlayerId)]
    
    # Pointing one search time completed
    toc = time()

    # Adding player name to the response
    playerInfo["batsman"] = bestMatch_PlayerName
    
    # Framing response model
    responseModel = {
        "response code" : 200,
        "search time"   : f"{toc-tic} seconds",
        "message"       : "success",
        "response body" : playerInfo
    }
    
    return responseModel

In [5]:
# Creating a function to make requests for data retrieval
def request(getPlayer=""):
    # Player name should be a string
    getPlayer = str(getPlayer)
    
    # Checking if the player name is valid or not
    if(getPlayer.replace(" ", "").strip() == ""):
        return f"Invalid Player Name"
    
    # Requesting to fetch the player info
    response = getClosestMatch(searchString = getPlayer)

    # Checking the response code
    if response["response code"] == 404:
        return f"ERROR: No record found"
    
    # Populating table by adding row
    o_row = response["response body"]["out"]
    i_row = response["response body"]["number_of_comments"]
    for r in o_row:
        try : specComments = i_row[0]['number_of_comments']
        except : specComments = ""
        break
    
    table.add_row([r["spectatorId"], r["out"], specComments, r["average"], response['response body']['onfield']['balls']])
               
    toBePrinted = f"\
    Batsman : {response['response body']['batsman']}\n\
    Strike Rate : {response['response body']['strike_rate']}\n\
    Total Runs : {response['response body']['onfield']['runs']}\n\
    "    
    print(toBePrinted)
    print(table)
    print(table.clear_rows())


In [6]:
# Making a request to fetch data
request(getPlayer="love dhoni")

    Batsman : MS Dhoni
    Strike Rate : 138.8022458
    Total Runs : 4450
    
+-------------+-----+--------------------+-------------+-----------------+
| SpectatorID | Out | Number Of Comments |   Average   | Number Of Balls |
+-------------+-----+--------------------+-------------+-----------------+
|      6      | 118 |         2          | 37.71186441 |       3206      |
+-------------+-----+--------------------+-------------+-----------------+
None
